# Day 3

In [2]:
import requests
from bs4 import BeautifulSoup

def get_aoc_problem(day, year=2023):
    url = f"https://adventofcode.com/{year}/day/{day}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # raises an exception for HTTP errors

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Assuming problem text is within an article tag (this might change)
        problem_text = soup.find('article').get_text()
        return problem_text
    except Exception as e:
        return f"Error fetching problem: {e}"

day = 3
problem_prompt = get_aoc_problem(day)
print(problem_prompt)

--- Day 3: Gear Ratios ---You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.
It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.
"Aaah!"
You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.
The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.
The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part num

In [48]:
with open('test.txt') as f:
    data = f.read()
    
lines = data.split('\n')[:-1]

for line in lines: 
    print(line)

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..


### Utility functions

In [2]:
def number_scanner(lines, number_locations, symbol_locations):
    '''
    lines is the input from the puzzle
    
    number_locations is a dictionary { key = (x,y) : digit }
    
    Indexing looks like:
    
    (0,0), (1,0), (2,0), (3,0), (4,0)
    (0,1), (1,1), (2,1), (3,1), (4,1)
    (0,2), (1,2), (2,2), (3,2), (4,2)
    (0,3), (1,3), (2,3), (3,3), (4,3)
    (0,4), (1,4), (2,4), (3,4), (4,4)
    
    returns:
    
    updated number locations
    '''
    
    for line_num in range(len(lines)):
        for pos_num in range(len(lines[0])):
            
            if lines[line_num][pos_num].isnumeric():
                number_locations[(pos_num, line_num)] = int(lines[line_num][pos_num])
            elif lines[line_num][pos_num] != '.':
                symbol_locations[(pos_num, line_num)] = lines[line_num][pos_num]
    
    return number_locations, symbol_locations

In [56]:
def sum_constructor(number_locations, symbol_locations):
    '''
    input is :
    
    number_locations - dictionary { key = (x,y) : digit }
    number_total - integer (current sum of all numbers)
    
    returns updated number_total
    
    '''
    number_total = 0
    
    indices_to_check = list(number_locations.keys())
    # print(indices_to_check)

    prev_index = (0,-1)
    number_str = ""
    
    for index in indices_to_check:
        
        # print(f'current index = {index}')
                
        if (index[0] - prev_index[0], index[1] - prev_index[1]) == (1, 0):
            # print(f"Indices {prev_index} and {index} are adjacent")
            number_str += str(number_locations[index])
        else:
            # print(f"Indices {prev_index} and {index} are not adjacent")
            if number_str != '':
                if check_number_radius(prev_index, number_str, symbol_locations):
                    # True --> add the number (has a symbol)
                    for i in range(len(number_str)):
                        number_locations_full[(prev_index[0]-i, prev_index[1])] = int(number_str)
                    number_total += int(number_str)
                    print(f'updated total = {number_total}')
                else:
                    print(f'updated total = {number_total}')
            
            # resets number string
            number_str = str(number_locations[index])
            
        prev_index = index
        # print(f'current number_total = {number_total}')
    
    print('last additions:')
    
    if check_number_radius(prev_index, number_str, symbol_locations):
        # True --> add the number (has a symbol)
        # print(f'number: {number_str} has an adjacent symbol... adding to total')
        number_total += int(number_str)
        print(f'updated total = {number_total}')
    else:
        pass
        print(f'number: {number_str} has NO adjacent symbols... NOT adding to total')
        print(f'updated total = {number_total}')
    
    
    # print(f'number_total = {number_total}') 
            
    return number_total

In [42]:
def check_number_radius(index, number_str, symbol_locations):
    
    num_len = len(number_str)
    
    min_x = max(0, index[0] - len(number_str))
    max_x = min(index[0] + 1, len(lines[0]) - 1)
                
    min_y = max(0, index[1] - 1)
    max_y = min(index[1] + 1, len(lines) - 1)
    
    for y in [min_y, max_y]:
        for x in range(min_x, max_x+1):
            if (x,y) in symbol_locations.keys():
                return True
    
    if max_y-min_y == 2:
        # check middle
        for x in [min_x,max_x]:
            if (x,min_y+1) in symbol_locations.keys():
                return True
        
    return False

### Part 1

In [58]:
number_locations = {}
number_locations_full = {}
symbol_locations = {}
number_total = 0

number_locations, symbol_locations = number_scanner(lines, number_locations, symbol_locations)

print(f'Final sum = {sum_constructor(number_locations, symbol_locations)}')

updated total = 467
updated total = 467
updated total = 502
updated total = 1135
updated total = 1752
updated total = 1752
updated total = 2344
updated total = 3099
updated total = 3763
last additions:
updated total = 4361
Final sum = 4361


### Part 2

In [55]:
for index in symbol_locations.keys():
    if symbol_locations[index] == '*':
        number_count = 0
        
        print(f'found a star at: {index}')
        
        indices_to_check = []
        
        min_x = max(0, index[0] - 1)
        max_x = min(index[0] + 1, len(lines[0]) - 1)

        min_y = max(0, index[1] - 1)
        max_y = min(index[1] + 1, len(lines) - 1)
        
        for y in [min_y, max_y]:
            line_count = 0
            for x in range(min_x, max_x+1):
                if (x,y) in number_locations.keys():
                    line_count += 1
                    indices_to_check.append((x,y))
                    
            if line_count == 1 or line_count == 3:
                print('exactly 1 number on row')
                number_count += 1
            elif line_count == 2 and (index[0], y) not in indices_to_check:
                print('exactly 2 numbers on row')
                number_count += 2
            elif line_count == 2 and (index[0], y) in indices_to_check:
                print('exactly 1 number on row')
                number_count += 1
            else:
                print('exactly 0 numbers on row')
                number_count += 0
                
                
        if max_y - min_y == 2:
            for x in [min_x, max_x]:
                if (x, min_y + 1) in number_locations.keys():
                    number_count += 1
        print(f'number count = {number_count}')            
        if number_count == 2:
            
            print(f'* at index {index} is a gear piece!')
                
                
        # print(f'indices to check: {indices_to_check} next to * at {index}')

found a star at: (3, 1)
exactly 1 number on row
exactly 1 number on row
number count = 2
* at index (3, 1) is a gear piece!
found a star at: (3, 4)
exactly 0 numbers on row
exactly 0 numbers on row
number count = 1
found a star at: (5, 8)
exactly 1 number on row
exactly 1 number on row
number count = 2
* at index (5, 8) is a gear piece!
